In [3]:
import os

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#GPU 사용
device = torch.device("cuda:0")

In [4]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
NVIDIA TITAN RTX


In [28]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/home/seclab_taewan/Taewan/NLP/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
using cached model. /home/seclab_taewan/Taewan/NLP/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [29]:
import pandas as pd
tag_data = pd.read_csv('./BIO_TAGGED.txt', sep = ' ', header=None)
tag_data.columns = ['word', 'bio_tag']

In [30]:
import math
# data[data.tag==math.nan]
tag_data.bio_tag.unique()

array(['O', 'PER_B', 'ADDR_B', 'ID_B', 'NUM_B', 'ADDR_I', 'NUM', 'BIR_B',
       'BIR_I'], dtype=object)

In [31]:
tag_data.loc[(tag_data['bio_tag'] == "PER_B"), 'bio_tag'] = 1
tag_data.loc[(tag_data['bio_tag'] == "ADDR_B"), 'bio_tag'] = 2
tag_data.loc[(tag_data['bio_tag'] == "ADDR_I"), 'bio_tag'] = 2
tag_data.loc[(tag_data['bio_tag'] == "ID_B"), 'bio_tag'] = 3
tag_data.loc[(tag_data['bio_tag'] == "NUM_B"), 'bio_tag'] = 4
tag_data.loc[(tag_data['bio_tag'] == "NUM"), 'bio_tag'] = 4
tag_data.loc[(tag_data['bio_tag'] == "BIR_B"), 'bio_tag'] = 5
tag_data.loc[(tag_data['bio_tag'] == "BIR_I"), 'bio_tag'] = 5
tag_data.loc[(tag_data['bio_tag'] == "O"), 'bio_tag'] = 6


data_list = []
for q, label in zip(tag_data['word'], tag_data['bio_tag']):
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [32]:
tag_data.bio_tag.unique()

array([6, 1, 2, 3, 4, 5], dtype=object)

In [33]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1955, random_state=42)

In [34]:
dataset_test = dataset_test[:-36]

print(len(dataset_train))
print(len(dataset_test))

1728
384


In [35]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.word = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.word[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [36]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [37]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/seclab_taewan/Taewan/NLP/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [38]:
data_train[0]

(array([   2, 4512, 6228, 7678,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(5, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 6)

In [39]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [40]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,   ##클래스 수##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [41]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model = BERTClassifier(bertmodel,  dr_rate=0.5)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/etc/anaconda3/envs/test/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [42]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        # token_ids = token_ids.long()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        # segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long().to(device)
        # label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        # token_ids = token_ids.long()
        token_ids = token_ids.long().to(device)
        # segment_ids = segment_ids.long()
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        # label = label.long()
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_41512/2776478663.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/27 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.439415454864502 train acc 0.140625
epoch 1 train acc 0.6290509259259259


/tmp/ipykernel_41512/2776478663.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/6 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9635416666666666


  0%|          | 0/27 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.14167103171348572 train acc 0.984375
epoch 5 train acc 0.9849537037037037


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 5 test acc 0.9635416666666666


In [43]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            tag_data.loc[(tag_data['bio_tag'] == "PER_B"), 'bio_tag'] = 1
            tag_data.loc[(tag_data['bio_tag'] == "ADDR_B"), 'bio_tag'] = 2
            tag_data.loc[(tag_data['bio_tag'] == "ADDR_I"), 'bio_tag'] = 2
            tag_data.loc[(tag_data['bio_tag'] == "ID_B"), 'bio_tag'] = 3
            tag_data.loc[(tag_data['bio_tag'] == "NUM_B"), 'bio_tag'] = 4
            tag_data.loc[(tag_data['bio_tag'] == "NUM"), 'bio_tag'] = 4
            tag_data.loc[(tag_data['bio_tag'] == "BIR_B"), 'bio_tag'] = 5
            tag_data.loc[(tag_data['bio_tag'] == "BIR_I"), 'bio_tag'] = 5
            tag_data.loc[(tag_data['bio_tag'] == "O"), 'bio_tag'] = 6

            if np.argmax(logits) == 1:
                test_eval.append("PER_B")
            elif np.argmax(logits) == 2:
                test_eval.append("ADDR")
            elif np.argmax(logits) == 3:
                test_eval.append("ID_B")
            elif np.argmax(logits) == 4:
                test_eval.append("NUM")
            elif np.argmax(logits) == 5:
                test_eval.append("BIR")
            elif np.argmax(logits) == 6:
                test_eval.append("O")

        print(">> 태그 : " + test_eval[0])

using cached model. /home/seclab_taewan/Taewan/NLP/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [44]:
end=1
while end ==1 :
  sentence = input("단어 입력 : ")
  if sentence == 0:
    break
  predict(sentence)
  print("\n")

단어 입력 : 김태완
>> 태그 : O


단어 입력 : 서울
>> 태그 : ADDR


단어 입력 : 960925
>> 태그 : ADDR


단어 입력 : 박준호
>> 태그 : O


단어 입력 : 흠
>> 태그 : O


단어 입력 : 특별시
>> 태그 : ADDR


단어 입력 : 106-81-43043
>> 태그 : ADDR




KeyboardInterrupt: Interrupted by user